# BO runs

In [1]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from scipy.stats import norm
from botorch.acquisition.analytic import ExpectedImprovement
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
import time

In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/cokes/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylel

load data from `prepare_Xy.ipynb`

In [2]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
y = np.reshape(y, (np.size(y), 1)) # for the GP
nb_data = np.size(y)
nb_data

69839

convert to torch tensors

In [3]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [4]:
X.size()

torch.Size([69839, 12])

In [5]:
y.size()

torch.Size([69839, 1])

In [6]:
X_unsqueezed = X.unsqueeze(1)

In [7]:
nb_data

69839

`ids_acquired[r, i]` will give ID of COF acquired during iteration `i` from run `r`.

In [8]:
# nb_iterations: # of COFs to acquire = # of iterations in BO
# nb_COFs_initialization: # of COFs to acquire at random to initiate the GP and BO.
def bo_run(nb_iterations, nb_COFs_initialization, which_acquisition, verbose=False):
    assert nb_iterations > nb_COFs_initialization
    
    # select initial COFs for training data randomly.
    # idea is to keep populating this ids_acquired and return it for analysis.
    ids_acquired = np.random.choice(np.arange((nb_data)), size=nb_COFs_initialization, replace=False)
    
    explore_exploit_balance = np.array([(np.NaN, np.NaN) for i in range(nb_iterations)])
    if which_acquisition != "EI":
        explore_exploit_balance = [] # don't bother storing NaNs

    # initialize acquired y, since it requires normalization
    y_acquired = y[ids_acquired]
    # standardize outputs
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i, end="\r")
        # construct and fit GP model
        model = SingleTaskGP(X[ids_acquired, :], y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # set up acquisition function
        if which_acquisition == "EI":
            acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
            
            # compute aquisition function at each COF in the database. 
#             batch_size = 35000 # need to do in batches to avoid mem issues
#             acquisition_values = torch.zeros((nb_data))
#             acquisition_values[:] = np.NaN # for safety
#             nb_batches = nb_data // batch_size
#             for ba in range(nb_batches+1):
#                 id_start = ba * batch_size
#                 id_end   = id_start + batch_size
#                 if id_end > nb_data:
#                     id_end = nb_data
#                 with torch.no_grad():
#                     acquisition_values[id_start:id_end] = acquisition_function.forward(X_unsqueezed[id_start:id_end])
#             assert acquisition_values.isnan().sum().item() == 0 # so that all are filled properly.
            with torch.no_grad(): # to avoid memory issues; we arent using the gradient...
                acquisition_values = acquisition_function.forward(X_unsqueezed) # runs out of memory
        elif which_acquisition == "max y_hat":
            with torch.no_grad():
                acquisition_values = model.posterior(X_unsqueezed).mean.squeeze()
        elif which_acquisition == "max sigma":
            with torch.no_grad():
                acquisition_values = model.posterior(X_unsqueezed).variance.squeeze()
        else:
            raise Exception("not a valid acquisition function")

        # select COF to acquire with maximal aquisition value, which is not in the acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, [id_max_aquisition]))
        assert np.size(ids_acquired) == i + 1
        
        # if EI, compute explore-exploit component.
        if which_acquisition == "EI":
            y_pred = model.posterior(X_unsqueezed[id_max_aquisition]).mean.squeeze().detach().numpy()
            sigma_pred = np.sqrt(model.posterior(X_unsqueezed[id_max_aquisition]).variance.squeeze().detach().numpy())
            
            y_max = y_acquired.max().item()
            
            z = (y_pred - y_max) / sigma_pred
            explore_term = sigma_pred * norm.pdf(z)
            exploit_term = (y_pred - y_max) * norm.cdf(z)
            
            assert np.isclose(explore_term + exploit_term, acquisition_values[id_max_aquisition].item())
            
#             print("explore term", explore_term)
#             print("exploit term", exploit_term)
#             print("my EI", explore_term + exploit_term)
#             print("auto EI", acquisition_values[id_max_aquisition].item())

            explore_exploit_balance[i] = (explore_term, exploit_term)

        # update y aquired; start over to normalize properly
        y_acquired = y[ids_acquired] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
        
        if verbose:
            print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition].item())
            print("\tbest y acquired:", y[ids_acquired].max().item())
        
    assert np.size(ids_acquired) == nb_iterations
    return ids_acquired, explore_exploit_balance

In [10]:
which_acquisition = "EI"
# which_acquisition = "max y_hat"
# which_acquisition = "max sigma"
nb_iterations = 20
nb_runs = 1
nb_COFs_initializations = {"EI": [5, 10, 15], "max y_hat": [10], "max sigma": [10]}

for nb_COFs_initialization in nb_COFs_initializations[which_acquisition]:
    print("# COFs in initialization:", nb_COFs_initialization)
    # store results here.
    bo_res = dict() 
    bo_res['nb_runs']                 = nb_runs
    bo_res['nb_iterations']           = nb_iterations
    bo_res['ids_acquired']            = []
    bo_res['explore_exploit_balance'] = []
    
    for r in range(nb_runs):
        print("\nRUN", r)
        t0 = time.time()
        
        ids_acquired, explore_exploit_balance = bo_run(bo_res['nb_iterations'], nb_COFs_initialization, which_acquisition)
        
        # store results from this run.
        bo_res['ids_acquired'].append(ids_acquired)
        bo_res['explore_exploit_balance'].append(explore_exploit_balance)
        
        print("took time t = ", (time.time() - t0) / 60, "min\n")
    
    # save results from all runs
    with open('bo_results_' + which_acquisition + "_initiate_with_{0}".format(nb_COFs_initialization) + '.pkl', 'wb') as file:
        pickle.dump(bo_res, file)
        
with open('bo_results_nb_COF_initializations.pkl', 'wb') as file:
    pickle.dump(nb_COFs_initializations, file)

# COFs in initialization: 5


RUN 0
took time t =  0.842117973168691 min
# COFs in initialization: 10


RUN 0
took time t =  1.011927890777588 min
# COFs in initialization: 15


RUN 0
took time t =  0.43527604738871256 min
